# Below cell must be run for any extraction

In [4]:
import inro.modeller as m
import os
import glob
import pandas as pd
import numpy as np
from copy import deepcopy

mm = m.Modeller()
eb = mm.emmebank
import inro.emme.matrix as _matrix

matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")  
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')
create_extra = m.Modeller().tool("inro.emme.data.extra_attribute.create_extra_attribute")
network_calc= m.Modeller().tool("inro.emme.network_calculation.network_calculator")
export_matrices = mm.tool("tmg.input_output.export_binary_matrix")  


def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"

def mapTripPurpose(purpose):
    purpose_list = purpose.split("-")
    purpose_o = purpose_list[0]
    purpose_d = purpose_list[1]
    work = ["PrimaryWork","SecondaryWork"]
    market = ["Market","JointMarket"]
    other = ["IndividualOther","JointOther"]
    if (purpose_o == "Home" or purpose_d == "Home") and (purpose_o in work or purpose_d in work):
        return "1 - HBW"
    elif (purpose_o == "Home" or purpose_d == "Home") and (purpose_o == "School" or purpose_d == "School"):
        return "2 - HBS"
    elif (purpose_o == 'WorkBasedBusiness' or purpose_d == 'WorkBasedBusiness'):
        return "3 - WBB"
    elif (purpose_d in market):
        return "4 - Market"
    elif (purpose_d in other):
        return "5 - Other"
    elif (purpose_o in market):
        return "4 - Market"
    elif (purpose_o in other):
        return "5 - Other"
    else:
        return "6 - Remaining"
def specificPurpose(purpose):
    purpose_list = purpose.split("-")
    purpose_o = purpose_list[0]
    purpose_d = purpose_list[1]
    work = ["PrimaryWork","SecondaryWork"]
    market_avoid_d = ['IndividualOther','JointOther','JointMarket']
    other_avoid_d = ['Market','JointOther','JointMarket']
    market_avoid_o = ['JointOther','JointMarket']
    other_avoid_o = ['JointOther','JointMarket']
    
    if (purpose_o in work and purpose_d == 'WorkBasedBusiness'):
        return "1 - WBB"
    elif (purpose_o == "Market" and purpose_d not in market_avoid_d) or (purpose_d == "Market" and purpose_o not in market_avoid_o):
        return "2 - Market"
    elif (purpose_o == "IndividualOther" and purpose_d not in other_avoid_d) or (purpose_d == "IndividualOther" and purpose_o not in other_avoid_o):
        return "3 - Other"
    else:
        return "4 - Remaining"

# User input in cell below

In [5]:
run_directory = r"C:\Users\BCrane\Documents\XTMF\Projects\Durham Update" #Path to XTMF Project Directory
run_folder = r"Durham Full Run 2016 - DRT Half Boarding Penalty - 20210519" # Name of run being processed
output_nme = "model_run_name_test" #Name of run that will be added to all processed outputs
input_pth = r"C:\Users\bcrane\Documents\Durham" #Directory where support files for output extraction are saved
output_pth = r"C:\Users\bcrane\Documents\Durham" #Directory where you want processed model outputs to be saved
zone_map = pd.read_csv(os.path.join(input_pth,"GTA_Durham_Zone_Mapping.csv")) # This file must be located in input_pth directory
scen_num = 400 #Scenario must have same zone system as run being extracted

# Full Output Extraction Here

In [7]:
run_path = os.path.join(run_directory,run_folder)
zone_map_min = zone_map[["DTAZ","pd_new","pd_old", "pd_tts"]]
scenario = eb.scenario(scen_num)
microsim_location = r"Microsim Results"
microsim_path = os.path.join(run_path, microsim_location)
#PORPOW Model
porpow_location = r"Validation\PoRPoW\Model"
porpow_path = os.path.join(run_path,porpow_location)
import_mat = "mf50"
first = True
for root,dirs,files in os.walk(porpow_path):
    for matrix_file in files:
        name = os.path.splitext(matrix_file)[0]
        matrix_number = int(import_mat[2:])
        import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name)
        porpow_mat = eb.matrix(import_mat).get_numpy_data(scenario)
        indices = eb.matrix(import_mat).get_data(scen_num).indices
        if first:
            df_full = pd.DataFrame(porpow_mat,indices[0],indices[1]).stack().reset_index().rename(columns = {"level_0": "TAZ_O",
                                                                                               "level_1": "TAZ_D",
                                                                                              0: name})
            first = False
        else:
            df_curr = pd.DataFrame(porpow_mat,indices[0],indices[1]).stack().reset_index().rename(columns = {"level_0": "TAZ_O",
                                                                                               "level_1": "TAZ_D",
                                                                                              0: name})
            df_full = df_full.merge(df_curr, on= ["TAZ_O","TAZ_D"])
            
df_full = df_full.merge(zone_map_min,left_on = "TAZ_O", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_o", "pd_old":"pd_old_o", "pd_tts":"pd_tts_o"})
df_full = df_full.merge(zone_map_min,left_on = "TAZ_D", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_d", "pd_old":"pd_old_d", "pd_tts":"pd_tts_d"})
df_full["All"] = df_full["GF"]+df_full["GP"]+df_full["MF"]+df_full["MP"]+df_full["PF"]+df_full["PP"]+df_full["SF"]+df_full["SP"]
all_pivot = pd.pivot_table(df_full[(df_full.pd_tts_o > 0) & (df_full.pd_tts_d > 0)], index = "pd_tts_o", columns = "pd_tts_d", values = ["All"], aggfunc = sum)
all_pivot.stack().to_csv(os.path.join(output_pth,"model_porpow_{}.csv".format(output_nme)))

#Licenses
persons_df = pd.read_csv(os.path.join(microsim_path,"persons.csv"))
households_df = pd.read_csv(os.path.join(microsim_path,"households.csv"))
persons_df = persons_df.merge(households_df,on="household_id")
persons_df = persons_df.merge(zone_map_min,left_on = "home_zone", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_home", "pd_old":"pd_old_home", "pd_tts":"pd_tts_home"})
pot_drivers = persons_df[persons_df["age"] >= 16]
license_summary = pd.pivot_table(pot_drivers, index = "pd_old_home", columns = "license", values = "weight_x",aggfunc = sum).fillna(0)
license_summary.to_csv(os.path.join(output_pth,"licensepdsummary_{}.csv".format(output_nme)))

#Auto Ownership
def reduce_drivers(drivers):
    if drivers <= 3:
        return drivers
    else:
        return 4
persons_df["license_num"] = persons_df["license"].map({True: 1, False: 0})
license_hh = pd.pivot_table(persons_df, index = "household_id", values = "license_num",aggfunc = sum).reset_index()
household_lic_df = households_df.merge(license_hh, on= "household_id").rename(columns = {"license_num": "drivers"})
household_lic_df = household_lic_df.merge(zone_map_min,left_on = "home_zone", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_home", "pd_old":"pd_old_home", "pd_tts":"pd_tts_home"})
household_lic_df["driver_mod"] = household_lic_df["drivers"].map(reduce_drivers)
household_lic_summary = pd.pivot_table(household_lic_df, index = ["driver_mod","pd_old_home"], columns = "vehicles", values = "weight", aggfunc = sum).fillna(0)
household_lic_summary.to_csv(os.path.join(output_pth,"householdvehiclesummary_{}.csv".format(output_nme)))

#Episode Generation
persons_df = pd.read_csv(os.path.join(microsim_path,"persons.csv"))
trips_df = pd.read_csv(os.path.join(microsim_path,"trips.csv"))
durham_trips = trips_df[((trips_df.o_zone >= 1000) & (trips_df.o_zone < 2000))| ((trips_df.d_zone >= 1000) & (trips_df.d_zone < 2000))]
durham_trips = durham_trips.merge(persons_df,on=["household_id","person_id"])
durham_trips = durham_trips[durham_trips.age >=11]
durham_trips["combined_act"] = durham_trips.o_act + "-" + durham_trips.d_act
durham_trips = durham_trips.merge(zone_map_min,left_on = "o_zone", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_o", "pd_old":"pd_old_o", "pd_tts":"pd_tts_o"})
durham_trips = durham_trips.merge(zone_map_min,left_on = "d_zone", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_d", "pd_old":"pd_old_d", "pd_tts":"pd_tts_d"})
durham_trips["Purpose"] = durham_trips["combined_act"].map(mapTripPurpose)
trip_summary = pd.pivot_table(durham_trips,values = "weight_x", index = ["pd_tts_o","pd_tts_d"], columns = "Purpose", aggfunc = sum, margins = True).fillna(0)

trip_summary.to_csv(os.path.join(output_pth,"dailytrips_episodegen_{}.csv".format(output_nme)))

#Location Choice
durham_trips["Purp2"] = durham_trips["combined_act"].map(specificPurpose)
trip_summary2 = pd.pivot_table(durham_trips,values = "weight_x", index = ["pd_tts_o","pd_tts_d"], columns = "Purp2", aggfunc = sum).fillna(0)
tripmode_df = pd.read_csv(os.path.join(microsim_path,"trip_modes.csv"))
durham_trips_time = tripmode_df.merge(durham_trips,how = "right", on = ['household_id', 'person_id', 'trip_id'])
durham_trips_time["weight"] = durham_trips_time["weight"]/10
durham_trips_am = durham_trips_time[(durham_trips_time.o_depart >= 360) & (durham_trips_time.o_depart < 540)]
durham_trips_pm = durham_trips_time[(durham_trips_time.o_depart >= 900) & (durham_trips_time.o_depart < 1140)]
trip_summary_am = pd.pivot_table(durham_trips_am,values = "weight", index = ["pd_tts_o","pd_tts_d"], columns = "Purp2", aggfunc = sum).fillna(0).reset_index()
trip_summary_pm = pd.pivot_table(durham_trips_pm,values = "weight", index = ["pd_tts_o","pd_tts_d"], columns = "Purp2", aggfunc = sum).fillna(0).reset_index()
trip_summary_am = trip_summary_am.rename(columns= {'1 - WBB': 'AM - 1 - WBB', '2 - Market': 'AM - 2 - Market', 
                                 '3 - Other': 'AM - 3 - Other', '4 - Remaining': 'AM - 4 - Remaining'})
trip_summary_pm = trip_summary_pm.rename(columns= {'1 - WBB': 'PM - 1 - WBB', '2 - Market': 'PM - 2 - Market', 
                                 '3 - Other': 'PM - 3 - Other', '4 - Remaining': 'PM - 4 - Remaining'})
trip_summary2 = trip_summary2.merge(trip_summary_am, on=['pd_tts_o', 'pd_tts_d'], how = "left").merge(trip_summary_pm, on=['pd_tts_o', 'pd_tts_d'], how = "left").fillna(0)

trip_summary2.to_csv(os.path.join(output_pth,"daily_ampm_locchoicesummary_{}.csv".format(output_nme)))

microsim_location = r"Microsim Results"
microsim_path = os.path.join(run_path, microsim_location)

tripmode_df = pd.read_csv(os.path.join(microsim_path,"trip_modes.csv"))

#GTA-Wide Mode Share
trips_gta = trips_df[(trips_df.o_zone < 6000) & (trips_df.d_zone < 6000)]
trips_gta_time = tripmode_df.merge(trips_gta,how = "right", on = ['household_id', 'person_id', 'trip_id'])
trips_gta_time["weight_x"] = trips_gta_time["weight_x"]/10
trips_gta_time["weight_z"] = trips_gta_time["weight_x"]*trips_gta_time["weight_y"]
pd.pivot_table(trips_gta_time,columns = "mode", values = "weight_z", aggfunc = sum).to_csv(os.path.join(output_pth,"Daily_GTATrips_Mode_{}.csv".format(output_nme)))

#Durham-Specific Mode Share
am_modeshare = pd.pivot_table(durham_trips_am,index = ["pd_tts_o","pd_tts_d"],columns = "mode", values = "weight", aggfunc = sum,margins = True).fillna(0)
pm_modeshare = pd.pivot_table(durham_trips_pm,index = ["pd_tts_o","pd_tts_d"],columns = "mode", values = "weight", aggfunc = sum,margins = True).fillna(0)
am_modeshare.to_csv(os.path.join(output_pth,"am_modeshare_{}.csv".format(output_nme)))
pm_modeshare.to_csv(os.path.join(output_pth,"pm_modeshare_{}.csv".format(output_nme)))

# PORPOW Outputs

In [4]:
run_path = os.path.join(run_directory,run_folder)
zone_map_min = zone_map[["DTAZ","pd_new","pd_old", "pd_tts"]]
scenario = eb.scenario(scen_num)
porpow_location = r"Validation\PoRPoW\Model"
porpow_path = os.path.join(run_path,porpow_location)
import_mat = "mf50"
first = True
for root,dirs,files in os.walk(porpow_path):
    for matrix_file in files:
        name = os.path.splitext(matrix_file)[0]
        matrix_number = int(import_mat[2:])
        import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name)
        porpow_mat = eb.matrix(import_mat).get_numpy_data(scenario)
        indices = eb.matrix(import_mat).get_data(scen_num).indices
        if first:
            df_full = pd.DataFrame(porpow_mat,indices[0],indices[1]).stack().reset_index().rename(columns = {"level_0": "TAZ_O",
                                                                                               "level_1": "TAZ_D",
                                                                                              0: name})
            first = False
        else:
            df_curr = pd.DataFrame(porpow_mat,indices[0],indices[1]).stack().reset_index().rename(columns = {"level_0": "TAZ_O",
                                                                                               "level_1": "TAZ_D",
                                                                                              0: name})
            df_full = df_full.merge(df_curr, on= ["TAZ_O","TAZ_D"])
df_full = df_full.merge(zone_map_min,left_on = "TAZ_O", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_o", "pd_old":"pd_old_o", "pd_tts":"pd_tts_o"})
df_full = df_full.merge(zone_map_min,left_on = "TAZ_D", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_d", "pd_old":"pd_old_d", "pd_tts":"pd_tts_d"})
df_full["All"] = df_full["GF"]+df_full["GP"]+df_full["MF"]+df_full["MP"]+df_full["PF"]+df_full["PP"]+df_full["SF"]+df_full["SP"]
all_pivot = pd.pivot_table(df_full[(df_full.pd_tts_o > 0) & (df_full.pd_tts_d > 0)], index = "pd_tts_o", columns = "pd_tts_d", values = ["All"], aggfunc = sum)
all_pivot.stack().to_csv(os.path.join(output_pth,"model_porpow_{}.csv".format(output_nme)))

# Licenses and Household Vehicle Ownership

In [3]:
run_path = os.path.join(run_directory,run_folder)
zone_map_min = zone_map[["DTAZ","pd_new","pd_old", "pd_tts"]]
microsim_location = r"Microsim Results"
microsim_path = os.path.join(run_path, microsim_location)
#Licenses
persons_df = pd.read_csv(os.path.join(microsim_path,"persons.csv"))
households_df = pd.read_csv(os.path.join(microsim_path,"households.csv"))
persons_df = persons_df.merge(households_df,on="household_id")
persons_df = persons_df.merge(zone_map_min,left_on = "home_zone", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_home", "pd_old":"pd_old_home", "pd_tts":"pd_tts_home"})
pot_drivers = persons_df[persons_df["age"] >= 16]
license_summary = pd.pivot_table(pot_drivers, index = "pd_old_home", columns = "license", values = "weight_x",aggfunc = sum).fillna(0)
license_summary.to_csv(os.path.join(output_pth,"licensepdsummary_{}.csv".format(output_nme)))
#Auto Ownership
def reduce_drivers(drivers):
    if drivers <= 3:
        return drivers
    else:
        return 4
persons_df["license_num"] = persons_df["license"].map({True: 1, False: 0})
license_hh = pd.pivot_table(persons_df, index = "household_id", values = "license_num",aggfunc = sum).reset_index()
household_lic_df = households_df.merge(license_hh, on= "household_id").rename(columns = {"license_num": "drivers"})
household_lic_df = household_lic_df.merge(zone_map_min,left_on = "home_zone", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_home", "pd_old":"pd_old_home", "pd_tts":"pd_tts_home"})
household_lic_df["driver_mod"] = household_lic_df["drivers"].map(reduce_drivers)
household_lic_summary = pd.pivot_table(household_lic_df, index = ["driver_mod","pd_old_home"], columns = "vehicles", values = "weight", aggfunc = sum).fillna(0)
household_lic_summary.to_csv(os.path.join(output_pth,"householdvehiclesummary_{}.csv".format(output_nme)))

# Episode Generation and Location Choice

In [3]:
run_path = os.path.join(run_directory,run_folder)
zone_map_min = zone_map[["DTAZ","pd_new","pd_old", "pd_tts"]]
microsim_location = r"Microsim Results"
microsim_path = os.path.join(run_path, microsim_location)

#Episode Generation
persons_df = pd.read_csv(os.path.join(microsim_path,"persons.csv"))
trips_df = pd.read_csv(os.path.join(microsim_path,"trips.csv"))
durham_trips = trips_df[((trips_df.o_zone >= 1000) & (trips_df.o_zone < 2000))| ((trips_df.d_zone >= 1000) & (trips_df.d_zone < 2000))]
durham_trips = durham_trips.merge(persons_df,on=["household_id","person_id"])
durham_trips = durham_trips[durham_trips.age >=11]
durham_trips["combined_act"] = durham_trips.o_act + "-" + durham_trips.d_act
durham_trips = durham_trips.merge(zone_map_min,left_on = "o_zone", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_o", "pd_old":"pd_old_o", "pd_tts":"pd_tts_o"})
durham_trips = durham_trips.merge(zone_map_min,left_on = "d_zone", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_d", "pd_old":"pd_old_d", "pd_tts":"pd_tts_d"})
durham_trips["Purpose"] = durham_trips["combined_act"].map(mapTripPurpose)
trip_summary = pd.pivot_table(durham_trips,values = "weight_x", index = ["pd_tts_o","pd_tts_d"], columns = "Purpose", aggfunc = sum, margins = True).fillna(0)

trip_summary.to_csv(os.path.join(output_pth,"dailytrips_episodegen_{}.csv".format(output_nme)))

#Location Choice
durham_trips["Purp2"] = durham_trips["combined_act"].map(specificPurpose)
trip_summary2 = pd.pivot_table(durham_trips,values = "weight_x", index = ["pd_tts_o","pd_tts_d"], columns = "Purp2", aggfunc = sum).fillna(0)
tripmode_df = pd.read_csv(os.path.join(microsim_path,"trip_modes.csv"))
durham_trips_time = tripmode_df.merge(durham_trips,how = "right", on = ['household_id', 'person_id', 'trip_id'])
durham_trips_time["weight"] = durham_trips_time["weight"]/10
durham_trips_am = durham_trips_time[(durham_trips_time.o_depart >= 360) & (durham_trips_time.o_depart < 540)]
durham_trips_pm = durham_trips_time[(durham_trips_time.o_depart >= 900) & (durham_trips_time.o_depart < 1140)]
trip_summary_am = pd.pivot_table(durham_trips_am,values = "weight", index = ["pd_tts_o","pd_tts_d"], columns = "Purp2", aggfunc = sum).fillna(0).reset_index()
trip_summary_pm = pd.pivot_table(durham_trips_pm,values = "weight", index = ["pd_tts_o","pd_tts_d"], columns = "Purp2", aggfunc = sum).fillna(0).reset_index()
trip_summary_am = trip_summary_am.rename(columns= {'1 - WBB': 'AM - 1 - WBB', '2 - Market': 'AM - 2 - Market', 
                                 '3 - Other': 'AM - 3 - Other', '4 - Remaining': 'AM - 4 - Remaining'})
trip_summary_pm = trip_summary_pm.rename(columns= {'1 - WBB': 'PM - 1 - WBB', '2 - Market': 'PM - 2 - Market', 
                                 '3 - Other': 'PM - 3 - Other', '4 - Remaining': 'PM - 4 - Remaining'})
trip_summary2 = trip_summary2.merge(trip_summary_am, on=['pd_tts_o', 'pd_tts_d'], how = "left").merge(trip_summary_pm, on=['pd_tts_o', 'pd_tts_d'], how = "left").fillna(0)

trip_summary2.to_csv(os.path.join(output_pth,"daily_ampm_locchoicesummary_{}.csv".format(output_nme)))


# GTA-Wide Mode Share

In [3]:
run_path = os.path.join(run_directory,run_folder)
zone_map_min = zone_map[["DTAZ","pd_new","pd_old", "pd_tts"]]
microsim_location = r"Microsim Results"
microsim_path = os.path.join(run_path, microsim_location)

trips_df = pd.read_csv(os.path.join(microsim_path,"trips.csv"))
tripmode_df = pd.read_csv(os.path.join(microsim_path,"trip_modes.csv"))
trips_gta = trips_df[(trips_df.o_zone < 6000) & (trips_df.d_zone < 6000)]
trips_gta_time = tripmode_df.merge(trips_gta,how = "right", on = ['household_id', 'person_id', 'trip_id'])
trips_gta_time["weight_x"] = trips_gta_time["weight_x"]/10
trips_gta_time["weight_z"] = trips_gta_time["weight_x"]*trips_gta_time["weight_y"]
pd.pivot_table(trips_gta_time,columns = "mode", values = "weight_z", aggfunc = sum).to_csv(os.path.join(output_pth,"Daily_GTATrips_Mode_{}.csv".format(output_nme)))

# Durham Specific Peak Period Mode Share

In [6]:
run_path = os.path.join(run_directory,run_folder)
zone_map_min = zone_map[["DTAZ","pd_new","pd_old", "pd_tts"]]
microsim_location = r"Microsim Results"
microsim_path = os.path.join(run_path, microsim_location)

persons_df = pd.read_csv(os.path.join(microsim_path,"persons.csv"))
trips_df = pd.read_csv(os.path.join(microsim_path,"trips.csv"))
tripmode_df = pd.read_csv(os.path.join(microsim_path,"trip_modes.csv"))

durham_trips = trips_df[((trips_df.o_zone >= 1000) & (trips_df.o_zone < 2000))| ((trips_df.d_zone >= 1000) & (trips_df.d_zone < 2000))]
durham_trips = durham_trips.merge(persons_df,on=["household_id","person_id"])
durham_trips = durham_trips[durham_trips.age >=11]
durham_trips = durham_trips.merge(zone_map_min,left_on = "o_zone", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_o", "pd_old":"pd_old_o", "pd_tts":"pd_tts_o"})
durham_trips = durham_trips.merge(zone_map_min,left_on = "d_zone", right_on ="DTAZ").drop(columns = "DTAZ").rename(columns = {"pd_new":"pd_new_d", "pd_old":"pd_old_d", "pd_tts":"pd_tts_d"})


durham_trips_time = tripmode_df.merge(durham_trips,how = "right", on = ['household_id', 'person_id', 'trip_id'])
durham_trips_time["weight"] = durham_trips_time["weight"]/10
durham_trips_am = durham_trips_time[(durham_trips_time.o_depart >= 360) & (durham_trips_time.o_depart < 540)]
durham_trips_pm = durham_trips_time[(durham_trips_time.o_depart >= 900) & (durham_trips_time.o_depart < 1140)]

am_modeshare = pd.pivot_table(durham_trips_am,index = ["pd_tts_o","pd_tts_d"],columns = "mode", values = "weight", aggfunc = sum,margins = True).fillna(0)
pm_modeshare = pd.pivot_table(durham_trips_pm,index = ["pd_tts_o","pd_tts_d"],columns = "mode", values = "weight", aggfunc = sum,margins = True).fillna(0)
am_modeshare.to_csv(os.path.join(output_pth,"am_modeshare_{}.csv".format(output_nme)))
pm_modeshare.to_csv(os.path.join(output_pth,"pm_modeshare_{}.csv".format(output_nme)))